### Spark Moive Recommendation
In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/)

## Part0: Environment setting up

In [3]:
dbutils.library.installPyPI("koalas")

Out[40]: True

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pyspark.sql import functions as F
%matplotlib inline
import os
import databricks.koalas as ks
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
movies_df = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings_df = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links_df = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags_df = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [8]:
movies_df.show(5)
ratings_df.show(5)
links_df.show(5)
tags_df.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [9]:
# get statistics for users
users_result = ratings_df.groupBy("userId").count()\
.orderBy('count', ascending=False)

fig = plt.figure()
tmp=sns.distplot(users_result.toPandas()['count'], hist=True)
plt.ylabel('percentage')
plt.xlabel('number of ratings')
tmp.set_title('Distribution of number of ratings (per user)')
display(tmp)

In [10]:
# get statistics for movies
movies_result=ratings_df.groupBy("movieId").count()\
.orderBy('count', ascending=False)

fig = plt.figure()
tmp=sns.distplot(movies_result.toPandas()['count'], color = 'burlywood', hist=True)
plt.ylabel('percentage')
plt.xlabel('number of ratings')
tmp.set_title('Distribution of number of ratings (per movie)')
display(tmp)

In [11]:
# analysis movies by content
movies_genres_temp=movies_df.where('genres is not null')\
.select('title', F.explode(F.split('genres', '\|'))\
.alias('genres'))

movies_genres=movies_genres_temp.groupBy('genres').count()\
.orderBy('count', ascending=False)\
.toPandas()

plt.figure(figsize=(60,10))
tmp=sns.barplot(x='genres', y='count', data=movies_genres)
tmp.set_title('Number of movies of each category')
display(tmp)

In [12]:
## get statistics for ratings
ratings_result=ratings_df.groupBy('rating').count()\
.withColumn('rating', ratings_df.rating.cast('float'))\
.toPandas()

fig = plt.figure()
tmp=sns.barplot(x='rating', y='count', data=ratings_result)
tmp.set_title('Number of movies of each rating')
display(tmp)

In [13]:
rated_list=ratings_df.select(ratings_df.movieId)\
.where('rating is not null')\
.withColumnRenamed('movieId', 'rated').distinct()

unrated_list=movies_df.join(rated_list, movies_df.movieId==rated_list.rated, "left_outer")\
.where('rated is null')

print('{} movies are unrated, {} movies are rated'.format(unrated_list.count(), rated_list.count()))

unrated_list.toPandas().fillna('No')

,movieId,title,genres,rated
0,1076,"Innocents, The (1961)",Drama|Horror|Thriller,No
1,2939,Niagara (1953),Drama|Thriller,No
2,3338,For All Mankind (1989),Documentary,No
3,3456,"Color of Paradise, The (Rang-e khoda) (1999)",Drama,No
4,4194,I Know Where I'm Going! (1945),Drama|Romance|War,No
5,5721,"Chosen, The (1981)",Drama,No
6,6668,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance,No
7,6849,Scrooge (1970),Drama|Fantasy|Musical,No
8,7020,Proof (1991),Comedy|Drama|Romance,No
9,7792,"Parallax View, The (1974)",Thriller,No


In [14]:
tmp1 = ratings_df.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings_df.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [15]:
tmp1 = sum(ratings_df.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings_df.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

### Spark SQL and OLAP

In [17]:
movies_df.registerTempTable("movies")
ratings_df.registerTempTable("ratings")
links_df.registerTempTable("links")
tags_df.registerTempTable("tags")

### Q1: The number of Users

In [19]:
%sql 
select count(distinct userID) as Number_of_user
from ratings

Number_of_user
610


### Q2: The number of Movies

In [21]:
%sql 
select count(distinct movieID) as Number_of_movie
from movies

Number_of_movie
9742


### Q3:  How many movies are rated by users? List movies not rated before

In [23]:
%sql
select title, genres
from movies
where movieId not in (
  select movieId
  from ratings)

title,genres
"Innocents, The (1961)",Drama|Horror|Thriller
Niagara (1953),Drama|Thriller
For All Mankind (1989),Documentary
"Color of Paradise, The (Rang-e khoda) (1999)",Drama
I Know Where I'm Going! (1945),Drama|Romance|War
"Chosen, The (1981)",Drama
"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
Scrooge (1970),Drama|Fantasy|Musical
Proof (1991),Comedy|Drama|Romance
"Parallax View, The (1974)",Thriller


In [24]:
%sql 
select count(distinct movieId) as Number_movies_are_rated_by_users
from ratings

Number_movies_are_rated_by_users
9724


### Q4: List Movie Genres

In [26]:
%sql
select distinct category
from movies
lateral view explode(split(genres, '[|]')) as category
order by category

category
(no genres listed)
Action
Adventure
Animation
Children
Comedy
Crime
Documentary
Drama
Fantasy


### Q5: Movie for Each Category

In [28]:
%sql
select category as Category, count(movieId) as number
from movies lateral view explode(split(genres, '[|]')) as category
group by category
order by number desc

Category,number
Drama,4361
Comedy,3756
Thriller,1894
Action,1828
Romance,1596
Adventure,1263
Crime,1199
Sci-Fi,980
Horror,978
Fantasy,779


In [29]:
%sql
select t.Category, concat_ws(',', collect_set(t.title)) as list_of_movies
from (
  select Category, title
  from movies
  lateral view explode(split(genres, '[|]')) as Category) as t
group by t.Category

Category list_of_movies Crime Stealing Rembrandt (Rembrandt) (2003),Innocent Man, An (1989),Girl with the Dragon Tattoo, The (Män som hatar kvinnor) (2009),The Gambler (2014),Undertow (2004),Sorrow (2015),Shadow Dancer (2012),Sherlock Holmes and Dr. Watson: Acquaintance (1979),21 Jump Street (2012),Battles Without Honor & Humanity (Jingi naki tatakai) (1973),Blitz (2011),Alpha Dog (2007),Micmacs (Micmacs à tire-larigot) (2009),Alex Cross (2012),Perfect Crime, The (Crimen Ferpecto) (Ferpect Crime) (2004),Hellsinki (Rööperi) (2009),Be Cool (2005),Kingsman: The Secret Service (2015),Dark Passage (1947),Fast Five (Fast and the Furious 5, The) (2011),Desperate Living (1977),Net, The (1995),Analyze That (2002),Life Eternal (2015),High Sierra (1941),Run All Night (2015),Turner & Hooch (1989),Corruptor, The (1999),New Police Story (Xin jing cha gu shi) (2004),No Country for Old Men (2007),Pledge, The (2001),Untraceable (2008),Boy A (2007),Carlito's Way (1993),Jane Austen's Mafia! (1998),Hobo with a Shotgun (2011),Die Hard (1988),Dead Man Walking (1995),Cobra (1986),Saw IV (2007),Man on a Ledge (2012),Thief of Paris, The (Le voleur) (1967),The Falcon and the Snowman (1985),Brick Mansions (2014),Boiler Room (2000),Furious 7 (2015),Batman Forever (1995),Short Film About Killing, A (Krótki film o zabijaniu) (1988),Flashback (1990),Unleashed (Danny the Dog) (2005),Crank: High Voltage (2009),Femme Nikita, La (Nikita) (1990),Cellular (2004),Sexy Beast (2000),Player, The (1992),Man Who Wasn't There, The (2001),Three Burials of Melquiades Estrada, The (2006),Angels with Dirty Faces (1938),Flickering Lights (Blinkende lygter) (2000),Papillon (1973),Band of Outsiders (Bande à part) (1964),Good Time (2017),Hamlet (1996),Himizu (2011),Godfather: Part II, The (1974),New Jersey Drive (1995),Herod's Law (Ley de Herodes, La) (2000),Our Lady of the Assassins (Virgen de los sicarios, La) (2000),One False Move (1992),Dark Portals: The Chronicles of Vidocq (Vidocq) (2001),F/X2 (a.k.a. F/X 2 - The Deadly Art of Illusion) (1991),Fresh (1994),Lawless (2012),Wild Things (1998),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Tower Heist (2011),General, The (1998),Another Thin Man (1939),Money Train (1995),Scarface (1932),Faster Pussycat! Kill! Kill! (1965),All About the Benjamins (2002),Prime Suspect 2 (1992),Investigation of a Citizen Above Suspicion (Indagine su un cittadino al di sopra di ogni sospetto) (1970),The Nice Guys (2016),Max Payne (2008),Adventures Of Sherlock Holmes And Dr. Watson: The Twentieth Century Approaches (1986),Police Academy 5: Assignment: Miami Beach (1988),Harry Brown (2009),Sherlock Holmes: Dressed to Kill (1946),Extreme Ops (2002),M (1931),Best Men (1997),Leaves of Grass (2009),Big Deal on Madonna Street (I Soliti Ignoti) (1958),House of Games (1987),Nine to Five (a.k.a. 9 to 5) (1980),Small Time Crooks (2000),Set It Off (1996),Ninja (2009),Tsotsi (2005),Suddenly (1954),Real McCoy, The (1993),Henry: Portrait of a Serial Killer (1986),Hangover, The (2009),Scarface (1983),Lethal Weapon (1987),Postman Always Rings Twice, The (1946),Fire with Fire (2012),Bus 174 (Ônibus 174) (2002),Seven-Per-Cent Solution, The (1976),Scorched (2003),You Only Live Once (1937),Get the Gringo (2012),Tokyo Tribe (2014),Office Space (1999),Shepherd: Border Patrol, The (2008),Grand Canyon (1991),Double, The (2011),By the Gun (2014),I Am a Fugitive from a Chain Gang (1932),Clear and Present Danger (1994),Munich (2005),Brothers Bloom, The (2008),Ocean's Eleven (a.k.a. Ocean's 11) (1960),Gone in 60 Seconds (2000),Trailer Park Boys (1999),Lord of War (2005),Lethal Weapon 3 (1992),Limey, The (1999),Yellow Sea, The (a.k.a. The Murderer) (Hwanghae) (2010),Midnight in the Garden of Good and Evil (1997),City of God (Cidade de Deus) (2002),Into the Blue (2005),Death of a President (2006),Mobsters (1991),Killing Them Softly (2012),Youth of the Beast (Yaju no seishun) (1963),Time Lapse (2014),Incognito (1997),Heartbreakers (2001),Baby Boy (2001),Gridlock'd (1997),R

## Part2: Spark ALS based approach for training model
We will use an Spark ML to predict the ratings, here reload "ratings.csv" using ``sc.textFile`` and then convert it to the form of (user, item, rating) tuples.

### Data Preprocessing

In [32]:
ratings_df.show()

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
 1| 70| 3.0|964982400|
 1| 101| 5.0|964980868|
 1| 110| 4.0|964982176|
 1| 151| 5.0|964984041|
 1| 157| 5.0|964984100|
 1| 163| 5.0|964983650|
 1| 216| 5.0|964981208|
 1| 223| 3.0|964980985|
 1| 231| 5.0|964981179|
 1| 235| 4.0|964980908|
 1| 260| 5.0|964981680|
 1| 296| 3.0|964982967|
 1| 316| 3.0|964982310|
 1| 333| 5.0|964981179|
 1| 349| 4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows

In [33]:
movie_ratings=ratings_df.drop('timestamp')

In [34]:
# Data type convert
from pyspark.sql.types import IntegerType, FloatType
movie_ratings = movie_ratings.withColumn("userId", movie_ratings["userId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("movieId", movie_ratings["movieId"].cast(IntegerType()))
movie_ratings = movie_ratings.withColumn("rating", movie_ratings["rating"].cast(FloatType()))

In [35]:
movie_ratings.show()

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 1| 1| 4.0|
 1| 3| 4.0|
 1| 6| 4.0|
 1| 47| 5.0|
 1| 50| 5.0|
 1| 70| 3.0|
 1| 101| 5.0|
 1| 110| 4.0|
 1| 151| 5.0|
 1| 157| 5.0|
 1| 163| 5.0|
 1| 216| 5.0|
 1| 223| 3.0|
 1| 231| 5.0|
 1| 235| 4.0|
 1| 260| 5.0|
 1| 296| 3.0|
 1| 316| 3.0|
 1| 333| 5.0|
 1| 349| 4.0|
+------+-------+------+
only showing top 20 rows

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [37]:
# import package
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder

In [38]:
#Create test and train set
(training,test)=movie_ratings.randomSplit([0.8,0.2], seed = 2020)

In [39]:
#Create ALS model
als = ALS(maxIter=5, rank=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [40]:
#Tune model using ParamGridBuilder
paramGrid = ParamGridBuilder()\
            .addGrid(als.regParam, [0.1, 0.01, 0.001])\
            .addGrid(als.maxIter, [3, 5, 10])\
            .addGrid(als.rank, [5, 10, 15])\
            .build()

In [41]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [42]:
# Build Cross validation 
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

In [43]:
#Fit ALS model to training data
cvModel = crossval.fit(training)
predictions = cvModel.transform(training)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
Root-mean-square error = 0.6402292718604988

In [44]:
#Extract best model from the tuning exercise using ParamGridBuilder
best_model = cvModel.bestModel
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

### Model testing
And finally, make a prediction and check the testing error.

In [46]:
#Generate predictions and evaluate using RMSE
predictions=best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [47]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:"+str(best_model._java_obj.parent().getRank())), 
print (" MaxIter:"+str(best_model._java_obj.parent().getMaxIter())), 
print (" RegParam:"+str(best_model._java_obj.parent().getRegParam())), 

RMSE = 0.889497730242029
**Best Model**
 Rank:5
 MaxIter:10
 RegParam:0.1
Out[25]: (None,)

In [48]:
predictions.show()

+------+-------+------+----------+
userId|movieId|rating|prediction|
+------+-------+------+----------+
 91| 471| 1.0| 2.9395921|
 372| 471| 3.0| 3.9969108|
 218| 471| 4.0| 2.5556598|
 57| 471| 3.0| 3.5694926|
 462| 471| 2.5| 2.7136428|
 273| 471| 5.0| 4.087789|
 287| 471| 4.5| 2.7965558|
 111| 1088| 3.0| 2.9841242|
 177| 1088| 3.5| 3.2341912|
 474| 1088| 3.5| 3.0003214|
 479| 1088| 4.0| 3.3455203|
 554| 1088| 5.0| 4.1957216|
 489| 1088| 4.5| 2.7748826|
 51| 1088| 4.0| 3.2923267|
 391| 1088| 1.0| 2.545455|
 42| 1088| 3.0| 3.613615|
 587| 1238| 4.0| 3.4879296|
 19| 1238| 3.0| 3.305425|
 462| 1238| 3.5| 4.218357|
 216| 1238| 5.0| 4.184975|
+------+-------+------+----------+
only showing top 20 rows

## Part3: Model apply and see the performance

In [50]:
alldata=best_model.transform(movie_ratings)
rmse = evaluator.evaluate(alldata)
print ("RMSE = "+str(rmse))

RMSE = 0.6959094422900263

In [51]:
alldata.registerTempTable("alldata")

In [52]:
%sql 
select * from alldata

userId,movieId,rating,prediction
191,148,5.0,4.9399667
133,471,4.0,3.0288007
597,471,2.0,3.828037
385,471,4.0,3.2121243
436,471,3.0,3.28026
602,471,4.0,3.4240687
91,471,1.0,2.9395921
409,471,3.0,3.311348
372,471,3.0,3.9969108
599,471,2.5,2.9150715


In [53]:
%sql 
select * 
from movies join alldata on movies.movieId=alldata.movieId

movieId,title,genres,userId,movieId,rating,prediction
148,"Awfully Big Adventure, An (1995)",Drama,191,148,5.0,4.9399667
471,"Hudsucker Proxy, The (1994)",Comedy,133,471,4.0,3.0288007
471,"Hudsucker Proxy, The (1994)",Comedy,597,471,2.0,3.828037
471,"Hudsucker Proxy, The (1994)",Comedy,385,471,4.0,3.2121243
471,"Hudsucker Proxy, The (1994)",Comedy,436,471,3.0,3.28026
471,"Hudsucker Proxy, The (1994)",Comedy,602,471,4.0,3.4240687
471,"Hudsucker Proxy, The (1994)",Comedy,91,471,1.0,2.9395921
471,"Hudsucker Proxy, The (1994)",Comedy,409,471,3.0,3.311348
471,"Hudsucker Proxy, The (1994)",Comedy,372,471,3.0,3.9969108
471,"Hudsucker Proxy, The (1994)",Comedy,599,471,2.5,2.9150715


In [54]:
#plot actual and predicted ratings for movie with most number of ratings
movie_rated_most=alldata.where('movieId = 356').select('rating', 'prediction').orderBy('rating')

RMSE_evaluator_rounded = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
print(RMSE_evaluator_rounded.evaluate(movie_rated_most))

0.7434929345077363

In [55]:
movie_rated_most=movie_rated_most.toPandas()
abs(movie_rated_most.prediction-movie_rated_most.rating).mean()

Out[89]: 0.5970807

In [56]:
fig = plt.figure()
plt.plot(movie_rated_most.index, movie_rated_most.rating, 'x', label='rating')
plt.plot(movie_rated_most.index, movie_rated_most.prediction, '.', label='predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for movie with 329 ratings')
display(fig)

In [57]:
#plot actual and predicted ratings for movie with 1 ratings
movie_rated_least=alldata.join(movies_result, movies_result.movieId==alldata.movieId, 'left')\
.where('count=1').select('rating', 'prediction').orderBy('rating')

print(RMSE_evaluator_rounded.evaluate(movie_rated_least))

0.07634598671342865

In [58]:
movie_rated_least=movie_rated_least.toPandas()

fig = plt.figure()
plt.plot(movie_rated_least.index, movie_rated_least.rating, '*', label='rating')
plt.plot(movie_rated_least.index, movie_rated_least.prediction, '-', label='predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for movies with 1 ratings')
display(fig)

In [59]:
#plot actual and predicted ratings for most active user
user_rate_most=alldata.where('userId = 414').select('rating', 'prediction').orderBy('rating')

print(RMSE_evaluator_rounded.evaluate(user_rate_most))

0.6093414727698108

In [60]:
user_rate_most=user_rate_most.toPandas()

fig = plt.figure()
plt.plot(user_rate_most.index, user_rate_most.rating, 'X', label='rating')
plt.plot(user_rate_most.index, user_rate_most.prediction, '.', label='predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for user rated 2698 movies')
display(fig)

In [61]:
#plot actual and predicted ratings for least active user
user_pool=users_result.where('count=20').toPandas()['userId'].tolist()
user_rate_least=alldata.where(alldata.movieId.isin(user_pool)).select('rating', 'prediction').orderBy('rating')

print(RMSE_evaluator_rounded.evaluate(user_rate_least))

0.7215395431640433

In [62]:
user_rate_least=user_rate_least.toPandas()


fig = plt.figure()
plt.plot(user_rate_least.index, user_rate_least.rating, 'x', label='rating')
plt.plot(user_rate_least.index, user_rate_least.prediction, '.', label='predictions')
plt.xlabel('index')
plt.ylabel('ratings')
plt.legend()
plt.title('Actual & predicted ratings for users rated 20 movies')
display(fig)

In [63]:
#plot average abs error by each movie and number of ratings
average_by_movie=alldata.groupBy('movieId')\
.agg(F.mean(F.abs(alldata.rating-alldata.prediction)).alias('MEBM'), F.count('rating').alias('number_of_users_rated'))\
.orderBy('MEBM', ascending=False)

In [64]:
tmp=average_by_movie.toPandas()
fig = plt.figure()
tmp=sns.scatterplot(x='number_of_users_rated', y='MEBM', data=tmp)
tmp.set_title('average abs error (per movies)')
display(tmp)

In [65]:
#plot average abs error by number of ratings
average_by_rating_number=alldata.join(movies_result, movies_result.movieId==alldata.movieId, 'left')\
.groupBy('count')\
.agg(F.mean(F.abs(alldata.rating-alldata.prediction)).alias('MEBC'))\
.orderBy('count', ascending=False)

In [66]:
tmp=average_by_rating_number.toPandas()
fig = plt.figure()
tmp=sns.scatterplot(x='count', y='MEBC', data=tmp)
tmp.set_title('average abs error by number of ratings')
display(tmp)

## Part4: Applications
### Recommend moive to users with id: 575, 232. 
you can choose some users to recommend the moives

In [68]:
userRecommend = best_model.recommendForAllUsers(10)

In [69]:
display(userRecommend)

userId,recommendations
471,"List(List(170355, 5.1592226), List(3379, 5.1592226), List(6818, 4.995792), List(8477, 4.7675037), List(27156, 4.706015), List(7025, 4.656068), List(5328, 4.6193542), List(6442, 4.6193542), List(3951, 4.6193542), List(5416, 4.6193542))"
463,"List(List(6818, 5.2831), List(167746, 5.123127), List(69524, 5.1149445), List(40, 5.0658236), List(138966, 5.0622716), List(26073, 5.0622716), List(179135, 5.0622716), List(184245, 5.0622716), List(7071, 5.0622716), List(84273, 5.0622716))"
496,"List(List(84847, 5.0428166), List(123, 4.8339357), List(8477, 4.799603), List(5075, 4.7701507), List(6818, 4.718129), List(7669, 4.6664944), List(148881, 4.6582603), List(1172, 4.616334), List(60943, 4.599704), List(59018, 4.599704))"
148,"List(List(170355, 4.9223213), List(3379, 4.9223213), List(60943, 4.8295546), List(59018, 4.8295546), List(5075, 4.8064837), List(136469, 4.800598), List(7481, 4.727048), List(33649, 4.7086196), List(44193, 4.622156), List(6666, 4.608422))"
540,"List(List(170355, 5.436522), List(3379, 5.436522), List(6818, 5.3690057), List(136469, 5.342836), List(84273, 5.3168273), List(7071, 5.3168273), List(74226, 5.3168273), List(26073, 5.3168273), List(78836, 5.3168273), List(179135, 5.3168273))"
392,"List(List(170355, 4.877714), List(3379, 4.877714), List(89904, 4.8729186), List(6818, 4.7937493), List(102217, 4.7466836), List(4794, 4.7373147), List(5292, 4.736286), List(5747, 4.7312174), List(3030, 4.708967), List(1279, 4.6262946))"
243,"List(List(3283, 6.1808667), List(40, 6.1193004), List(4846, 6.1130157), List(67618, 6.090781), List(72171, 6.0813055), List(183897, 6.028741), List(51931, 6.0269623), List(3086, 5.992087), List(4256, 5.931812), List(147382, 5.9184594))"
31,"List(List(6818, 5.837201), List(70946, 5.681199), List(6732, 5.5443754), List(8477, 5.5411916), List(33649, 5.499454), List(5666, 5.378515), List(148881, 5.313654), List(25771, 5.23367), List(6461, 5.1617603), List(5466, 5.15401))"
516,"List(List(47423, 5.1373506), List(70946, 5.1115975), List(33649, 5.008463), List(112804, 5.007616), List(2563, 4.944204), List(391, 4.9298816), List(170355, 4.904602), List(3379, 4.904602), List(7748, 4.861166), List(4429, 4.8556337))"
580,"List(List(72171, 5.8612065), List(183897, 5.5348086), List(136469, 5.377276), List(167746, 5.1846724), List(170705, 5.1621943), List(5075, 5.100952), List(5137, 5.0989904), List(3837, 5.0884457), List(188751, 5.086501), List(174909, 5.086501))"


In [70]:
user_recommendation = userRecommend.to_koalas()
user_recommendation.head()

,userId,recommendations
0,471,"[(170355, 5.159222602844238), (3379, 5.1592226..."
1,463,"[(6818, 5.283100128173828), (167746, 5.1231269..."
2,496,"[(84847, 5.042816638946533), (123, 4.833935737..."
3,148,"[(170355, 4.922321319580078), (3379, 4.9223213..."
4,540,"[(170355, 5.436522006988525), (3379, 5.4365220..."


In [71]:
movies_koalas = movies_df.to_koalas()

In [72]:
def movie_recommendation(user_recommendation, userId, movies_koalas):
  rec_movieId = []
  for item in user_recommendation.loc['userId' == userId][1]:
    rec_movieId.append(item[0])
  return movies_koalas.loc[movies_koalas.movieId.isin(rec_movieId)]

In [73]:
movie_recommendation(user_recommendation, 575, movies_koalas)

,movieId,title,genres
2523,3379,On the Beach (1959),Drama
2947,3951,Two Family House (2000),Drama
3807,5328,Rain (2001),Drama|Romance
3852,5416,Cherish (2002),Comedy|Drama|Thriller
4390,6442,Belle époque (1992),Comedy|Romance
4590,6818,Come and See (Idi i smotri) (1985),Drama|War
4708,7025,"Midnight Clear, A (1992)",Drama|War
5202,8477,"Jetée, La (1962)",Romance|Sci-Fi
5621,27156,Neon Genesis Evangelion: The End of Evangelion...,Action|Animation|Drama|Fantasy|Sci-Fi
9489,170355,Mulholland Dr. (1999),Drama|Mystery|Romance


In [74]:
movie_recommendation(user_recommendation, 232, movies_koalas)

,movieId,title,genres
2523,3379,On the Beach (1959),Drama
2947,3951,Two Family House (2000),Drama
3807,5328,Rain (2001),Drama|Romance
3852,5416,Cherish (2002),Comedy|Drama|Thriller
4390,6442,Belle époque (1992),Comedy|Romance
4590,6818,Come and See (Idi i smotri) (1985),Drama|War
4708,7025,"Midnight Clear, A (1992)",Drama|War
5202,8477,"Jetée, La (1962)",Romance|Sci-Fi
5621,27156,Neon Genesis Evangelion: The End of Evangelion...,Action|Animation|Drama|Fantasy|Sci-Fi
9489,170355,Mulholland Dr. (1999),Drama|Mystery|Romance


### Find the similar moives for moive with id: 463, 471
You can find the similar moives based on the ALS results

In [76]:
item_factors = best_model.itemFactors
movie_factors = item_factors.to_koalas()
movie_factors.head()

,id,features
0,10,"[-0.36515158, 0.07847207, 1.0054493, -1.649805..."
1,20,"[-0.71949834, 0.8905786, 1.267864, -0.814857, ..."
2,30,"[-1.5475146, -0.44531843, 1.6068691, 0.4473641..."
3,40,"[-0.9048903, 0.4967277, 0.7463572, -2.3556798,..."
4,50,"[-1.1755126, 0.032277633, 1.519145, -1.423313,..."


In [77]:
def similar_movies(features, movieId):

  try: 
    target_id_feature = movie_factors.loc[movie_factors.id == movieId].features.to_numpy()[0]
  except:
    return 'There is no movie with id ' + str(movieId)

  similarities = []
  for feature in movie_factors['features'].to_numpy():
    similarity = np.dot(target_id_feature,feature)/(np.linalg.norm(target_id_feature) * np.linalg.norm(feature))
    similarities.append(similarity)
    
  ks_similarity = ks.DataFrame({'similarity' : similarities}, index = movie_factors.id.to_numpy())
  # top 11 similar movies contain the movie itself with similarity = 1, so I need to remove it. 
  top_11 = ks_similarity.sort_values(by = ['similarity'], ascending = False).head(11)
  joint = top_11.merge(movies_koalas, left_index=True, right_on = 'movieId', how = 'inner')
  joint.sort_values(by = ['similarity'], ascending = False,inplace = True)
  joint.reset_index(inplace = True)
  # take top 10 similar movies
  return joint.loc[1:,['movieId','title','genres','similarity']]

In [78]:
similar_movies(features = movie_factors['features'], movieId = 463)

Out[63]: 'There is no movie with id 463'

In [79]:
similar_movies(features = movie_factors['features'], movieId = 471)

,movieId,title,genres,similarity
1,3451,Guess Who's Coming to Dinner (1967),Drama,0.998200
2,6218,Bend It Like Beckham (2002),Comedy|Drama|Romance,0.997120
3,72,Kicking and Screaming (1995),Comedy|Drama,0.997102
4,534,Shadowlands (1993),Drama|Romance,0.996873
5,7624,School Ties (1992),Drama,0.995378
6,3350,"Raisin in the Sun, A (1961)",Drama,0.994899
7,1096,Sophie's Choice (1982),Drama,0.994833
8,27873,Metallica: Some Kind of Monster (2004),Documentary,0.994809
9,6002,Love Liza (2002),Drama,0.994706
10,31694,Bride & Prejudice (2004),Comedy|Musical|Romance,0.993139
